Простой пример: Рассылка новостей подписчикам (Push-модель)
Издатель (Субъект) публикует новости, а подписчики (Наблюдатели) получают их и выводят на экран.

In [1]:
from abc import ABC, abstractmethod
from typing import List, Any

# 1. Observer Interface (Интерфейс Наблюдателя)
class Observer(ABC):
    @abstractmethod
    def update(self, message: Any):
        """Метод, вызываемый Субъектом для уведомления."""
        pass

# 2. Subject Interface (Интерфейс Субъекта)
class Subject(ABC):
    @abstractmethod
    def attach(self, observer: Observer):
        """Подписывает наблюдателя."""
        pass

    @abstractmethod
    def detach(self, observer: Observer):
        """Отписывает наблюдателя."""
        pass

    @abstractmethod
    def notify(self, message: Any):
        """Уведомляет всех подписчиков."""
        pass

# 3. Concrete Subject (Конкретный Субъект - Новостное Агентство)
class NewsAgency(Subject):
    """Издатель новостей."""
    def __init__(self, name: str):
        self._name = name
        self._observers: List[Observer] = []
        self._latest_news: str | None = None
        print(f"NewsAgency '{self._name}' created.")

    def attach(self, observer: Observer):
        print(f"{self._name}: Attaching observer {type(observer).__name__}.")
        if observer not in self._observers:
            self._observers.append(observer)

    def detach(self, observer: Observer):
        print(f"{self._name}: Detaching observer {type(observer).__name__}.")
        try:
            self._observers.remove(observer)
        except ValueError:
            print(f"{self._name}: Observer not found.")

    def notify(self, message: Any):
        """Уведомляет всех подписчиков (Push-модель - передаем данные)."""
        print(f"{self._name}: Notifying observers about news: '{message}'")
        # Создаем копию списка на случай, если наблюдатель отпишется во время уведомления
        for observer in list(self._observers):
            print(f"  -> Notifying {type(observer).__name__}...")
            observer.update(message)

    def publish_news(self, news: str):
        """Публикует новость и уведомляет подписчиков."""
        print(f"\n{self._name}: Publishing news -> '{news}'")
        self._latest_news = news
        self.notify(self._latest_news) # Уведомляем с новыми данными

# 4. Concrete Observers (Конкретные Наблюдатели - Читатели)
class NewsReader(Observer):
    """Читатель, который просто выводит новость."""
    def __init__(self, name: str):
        self._name = name
        print(f"NewsReader '{self._name}' created.")

    def update(self, message: Any):
        print(f"  Reader '{self._name}': Received news update <- '{message}'")

class NewsChannel(Observer):
    """Новостной канал, который может по-другому реагировать."""
    def __init__(self, channel_name: str):
        self._channel_name = channel_name
        print(f"NewsChannel '{self._channel_name}' created.")

    def update(self, message: Any):
        print(f"  Channel '{self._channel_name}': Broadcasting news update: '{message.upper()}'") # Форматирует новость


# 5. Client Code
if __name__ == "__main__":
    # Создаем Субъекта
    agency = NewsAgency("Global News")

    # Создаем Наблюдателей
    reader1 = NewsReader("Alice")
    reader2 = NewsReader("Bob")
    channel1 = NewsChannel("News TV")

    # Подписываем Наблюдателей
    agency.attach(reader1)
    agency.attach(reader2)
    agency.attach(channel1)

    # Субъект публикует новость -> все подписчики уведомляются
    agency.publish_news("Important Breaking News!")

    # Один Наблюдатель отписывается
    agency.detach(reader2)

    # Субъект публикует другую новость
    agency.publish_news("Weather forecast: Sunny")

# Вывод:
# NewsAgency 'Global News' created.
# NewsReader 'Alice' created.
# NewsReader 'Bob' created.
# NewsChannel 'News TV' created.
# Global News: Attaching observer NewsReader.
# Global News: Attaching observer NewsReader.
# Global News: Attaching observer NewsChannel.
#
# Global News: Publishing news -> 'Important Breaking News!'
# Global News: Notifying observers about news: 'Important Breaking News!'
#   -> Notifying NewsReader...
#   Reader 'Alice': Received news update <- 'Important Breaking News!'
#   -> Notifying NewsReader...
#   Reader 'Bob': Received news update <- 'Important Breaking News!'
#   -> Notifying NewsChannel...
#   Channel 'News TV': Broadcasting news update: 'IMPORTANT BREAKING NEWS!'
# Global News: Detaching observer NewsReader.
#
# Global News: Publishing news -> 'Weather forecast: Sunny'
# Global News: Notifying observers about news: 'Weather forecast: Sunny'
#   -> Notifying NewsReader...
#   Reader 'Alice': Received news update <- 'Weather forecast: Sunny'
#   -> Notifying NewsChannel...
#   Channel 'News TV': Broadcasting news update: 'WEATHER FORECAST: SUNNY'

NewsAgency 'Global News' created.
NewsReader 'Alice' created.
NewsReader 'Bob' created.
NewsChannel 'News TV' created.
Global News: Attaching observer NewsReader.
Global News: Attaching observer NewsReader.
Global News: Attaching observer NewsChannel.

Global News: Publishing news -> 'Important Breaking News!'
Global News: Notifying observers about news: 'Important Breaking News!'
  -> Notifying NewsReader...
  Reader 'Alice': Received news update <- 'Important Breaking News!'
  -> Notifying NewsReader...
  Reader 'Bob': Received news update <- 'Important Breaking News!'
  -> Notifying NewsChannel...
  Channel 'News TV': Broadcasting news update: 'IMPORTANT BREAKING NEWS!'
Global News: Detaching observer NewsReader.

Global News: Publishing news -> 'Weather forecast: Sunny'
Global News: Notifying observers about news: 'Weather forecast: Sunny'
  -> Notifying NewsReader...
  Reader 'Alice': Received news update <- 'Weather forecast: Sunny'
  -> Notifying NewsChannel...
  Channel 'News T

Сложный пример: Мониторинг Температуры (Pull-модель и слабая связь)
Датчик температуры (Субъект) измеряет температуру. Различные устройства (дисплей, система оповещения - Наблюдатели) подписываются на него. Используется Pull-модель (Наблюдатели сами запрашивают данные) и слабые ссылки для управления подписками.

In [2]:
import time
import random
import weakref # Используем слабые ссылки
from abc import ABC, abstractmethod
from typing import Set, Any

# --- Observer Interface ---
class TemperatureObserver(ABC):
    @abstractmethod
    def update(self): # Pull-модель: нет аргументов
        """Метод уведомления."""
        pass

# --- Subject Interface ---
class TemperatureSubject(ABC):
    @abstractmethod
    def attach(self, observer: TemperatureObserver): pass
    @abstractmethod
    def detach(self, observer: TemperatureObserver): pass
    @abstractmethod
    def notify(self): pass
    @abstractmethod
    def get_temperature(self) -> float: pass # Метод для Pull-модели

# --- Concrete Subject ---
class TemperatureSensor(TemperatureSubject):
    """Датчик температуры."""
    def __init__(self, location: str):
        self._location = location
        # Используем weakref.WeakSet для автоматического удаления "мертвых" наблюдателей
        self._observers: Set[weakref.ReferenceType[TemperatureObserver]] = weakref.WeakSet()
        self._temperature: float = 20.0 + random.uniform(-2, 2)
        print(f"TemperatureSensor at '{self._location}' initialized. Current temp: {self._temperature:.1f}°C")

    def attach(self, observer: TemperatureObserver):
        print(f"Sensor '{self._location}': Attaching observer {type(observer).__name__}")
        self._observers.add(observer) # WeakSet сам управляет ссылками

    def detach(self, observer: TemperatureObserver):
        print(f"Sensor '{self._location}': Detaching observer {type(observer).__name__}")
        try:
            self._observers.remove(observer)
        except KeyError:
            print(f"Sensor '{self._location}': Observer not found or already removed.")

    def notify(self):
        """Уведомляет активных подписчиков."""
        print(f"Sensor '{self._location}': Notifying observers...")
        # Итерируемся по копии, чтобы избежать проблем при изменении set во время итерации
        for observer_ref in list(self._observers):
             # Проверяем, жив ли объект наблюдателя
             observer = observer_ref # В WeakSet хранятся сами объекты, а не ссылки
             if observer: # Проверка, что объект еще не собран GC
                print(f"  -> Notifying {type(observer).__name__}...")
                observer.update() # Вызываем update без данных
             else:
                print(f"  -> Observer reference is dead, skipping.")
                # WeakSet сам удалит мертвую ссылку

    def _measure_temperature(self):
        """Имитация измерения температуры."""
        change = random.uniform(-1.5, 1.5)
        # Ограничим температуру для примера
        self._temperature = max(10.0, min(30.0, self._temperature + change))
        print(f"\nSensor '{self._location}': Measured new temperature: {self._temperature:.1f}°C")
        self.notify() # Уведомляем после измерения

    def get_temperature(self) -> float:
        """Предоставляет температуру для Pull-модели."""
        print(f"Sensor '{self._location}': Providing temperature ({self._temperature:.1f}°C)")
        return self._temperature

    def run_simulation(self, steps: int):
        """Запускает имитацию измерений."""
        print(f"\n--- Running simulation for sensor '{self._location}' ({steps} steps) ---")
        for i in range(steps):
            print(f"-- Step {i+1}/{steps} --")
            self._measure_temperature()
            time.sleep(0.5) # Пауза между измерениями
        print("--- Simulation ended ---")


# --- Concrete Observers ---
class DisplayScreen(TemperatureObserver):
    """Дисплей, показывающий текущую температуру."""
    def __init__(self, name: str, sensor: TemperatureSensor):
        self._name = name
        # Наблюдателю нужна ссылка на Субъекта для Pull-модели
        self._sensor = sensor
        print(f"DisplayScreen '{self._name}' created.")

    def update(self):
        # Получив уведомление, запрашиваем данные
        current_temp = self._sensor.get_temperature()
        print(f"  Display '{self._name}': Updated temperature display to {current_temp:.1f}°C")

class AlertSystem(TemperatureObserver):
    """Система оповещения, срабатывающая при критических температурах."""
    def __init__(self, threshold: float, sensor: TemperatureSensor):
        self._threshold = threshold
        self._sensor = sensor
        self._alert_active = False
        print(f"AlertSystem created (threshold: {self._threshold}°C).")

    def update(self):
        current_temp = self._sensor.get_temperature()
        if current_temp > self._threshold and not self._alert_active:
            print(f"  !!! ALERT SYSTEM: Temperature {current_temp:.1f}°C EXCEEDS threshold {self._threshold}°C !!!")
            self._alert_active = True
        elif current_temp <= self._threshold and self._alert_active:
            print(f"  Alert System: Temperature {current_temp:.1f}°C back to normal.")
            self._alert_active = False
        # else: Не делаем ничего, если состояние не изменилось


# --- Client Code ---
if __name__ == "__main__":
    # Создаем Субъекта
    living_room_sensor = TemperatureSensor("Living Room")

    # Создаем Наблюдателей, передавая им Субъекта
    display1 = DisplayScreen("Main Display", living_room_sensor)
    alert1 = AlertSystem(25.0, living_room_sensor) # Порог срабатывания 25°C

    # Подписываем Наблюдателей
    living_room_sensor.attach(display1)
    living_room_sensor.attach(alert1)

    # Запускаем имитацию
    living_room_sensor.run_simulation(5)

    # Имитируем удаление одного из наблюдателей (например, дисплей выключили)
    print("\n--- Simulating DisplayScreen removal ---")
    del display1 # Удаляем сильную ссылку, weakref в сенсоре должен это учесть
    import gc
    gc.collect() # Принудительно запускаем сборщик мусора (не обязательно в реальном коде)

    # Продолжаем имитацию
    living_room_sensor.run_simulation(3)

# Примерный вывод (температуры будут случайными):
# TemperatureSensor at 'Living Room' initialized. Current temp: 19.8°C
# DisplayScreen 'Main Display' created.
# AlertSystem created (threshold: 25.0°C).
# Sensor 'Living Room': Attaching observer DisplayScreen
# Sensor 'Living Room': Attaching observer AlertSystem
#
# --- Running simulation for sensor 'Living Room' (5 steps) ---
# -- Step 1/5 --
# Sensor 'Living Room': Measured new temperature: 20.5°C
# Sensor 'Living Room': Notifying observers...
#   -> Notifying DisplayScreen...
# Sensor 'Living Room': Providing temperature (20.5°C)
#   Display 'Main Display': Updated temperature display to 20.5°C
#   -> Notifying AlertSystem...
# Sensor 'Living Room': Providing temperature (20.5°C)
# -- Step 2/5 --
# Sensor 'Living Room': Measured new temperature: 21.8°C
# Sensor 'Living Room': Notifying observers...
#   -> Notifying DisplayScreen...
# Sensor 'Living Room': Providing temperature (21.8°C)
#   Display 'Main Display': Updated temperature display to 21.8°C
#   -> Notifying AlertSystem...
# Sensor 'Living Room': Providing temperature (21.8°C)
# -- Step 3/5 --
# Sensor 'Living Room': Measured new temperature: 23.1°C
# ... (уведомления) ...
# -- Step 4/5 --
# Sensor 'Living Room': Measured new temperature: 24.5°C
# ... (уведомления) ...
# -- Step 5/5 --
# Sensor 'Living Room': Measured new temperature: 25.8°C
# Sensor 'Living Room': Notifying observers...
#   -> Notifying DisplayScreen...
# Sensor 'Living Room': Providing temperature (25.8°C)
#   Display 'Main Display': Updated temperature display to 25.8°C
#   -> Notifying AlertSystem...
# Sensor 'Living Room': Providing temperature (25.8°C)
#   !!! ALERT SYSTEM: Temperature 25.8°C EXCEEDS threshold 25.0°C !!!
# --- Simulation ended ---
#
# --- Simulating DisplayScreen removal ---
#
# --- Running simulation for sensor 'Living Room' (3 steps) ---
# -- Step 1/3 --
# Sensor 'Living Room': Measured new temperature: 25.1°C
# Sensor 'Living Room': Notifying observers...
#   -> Notifying AlertSystem...
# Sensor 'Living Room': Providing temperature (25.1°C)
# -- Step 2/3 --
# Sensor 'Living Room': Measured new temperature: 24.0°C
# Sensor 'Living Room': Notifying observers...
#   -> Notifying AlertSystem...
# Sensor 'Living Room': Providing temperature (24.0°C)
#   Alert System: Temperature 24.0°C back to normal.
# -- Step 3/3 --
# Sensor 'Living Room': Measured new temperature: 23.5°C
# Sensor 'Living Room': Notifying observers...
#   -> Notifying AlertSystem...
# Sensor 'Living Room': Providing temperature (23.5°C)
# --- Simulation ended ---

TemperatureSensor at 'Living Room' initialized. Current temp: 21.7°C
DisplayScreen 'Main Display' created.
AlertSystem created (threshold: 25.0°C).
Sensor 'Living Room': Attaching observer DisplayScreen
Sensor 'Living Room': Attaching observer AlertSystem

--- Running simulation for sensor 'Living Room' (5 steps) ---
-- Step 1/5 --

Sensor 'Living Room': Measured new temperature: 21.1°C
Sensor 'Living Room': Notifying observers...
  -> Notifying DisplayScreen...
Sensor 'Living Room': Providing temperature (21.1°C)
  Display 'Main Display': Updated temperature display to 21.1°C
  -> Notifying AlertSystem...
Sensor 'Living Room': Providing temperature (21.1°C)
-- Step 2/5 --

Sensor 'Living Room': Measured new temperature: 21.4°C
Sensor 'Living Room': Notifying observers...
  -> Notifying DisplayScreen...
Sensor 'Living Room': Providing temperature (21.4°C)
  Display 'Main Display': Updated temperature display to 21.4°C
  -> Notifying AlertSystem...
Sensor 'Living Room': Providing temper